In [1]:
 current_dir = "gdrive/My Drive/Colab Notebooks/Thesis/gold_standard"


import os
import glob
from google.colab import drive
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Mount Google Drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
os.chdir(current_dir)

#### Pickling

In [ ]:
# @title
subjects = []


# Get the current working directory
current_directory = os.getcwd()

# List all .txt files in the current directory and process each file
for file in glob.glob(os.path.join(current_directory, '*.txt')):
    with open(file, 'r') as f:
        file_content = f.read()
    filename_without_extension = os.path.basename(file).split('.txt')[0]
    split_filename = filename_without_extension.split('_')
    name = split_filename[0]
    score = float(split_filename[1])
    data = [name, score, file_content]
    subjects.append(data)

df = pd.DataFrame(subjects, columns=['name', 'score', 'text'])

# Show the DataFrame
print(df.head())

df.to_pickle("gold_psy.pkl")

In [3]:
# @title
# Unpickle the DataFrame
gold_standard = pd.read_pickle("gold_psy.pkl")
print(gold_standard)


       name  score                                               text
0     wayne   40.0  \nmy very first contemplation of criminal beha...
1      carl   36.0  I had a statutory rape on a minor fifteen uh t...
2     bundy   34.0  \nyes that's true\n\ni grew up in a wonderful ...
3  bernardo   35.0  \npaul jason teal aka paul bernardo\n\n\nyeah ...
4    dahmer   23.0  \nhey you're lucky you came up on a\n\n\nday w...
5      gacy   27.0  \n\nwell that's hogwash yeah that's strictly h...
6    kemper   26.0  \nwell i'm not an expert i'm not an authority ...
7     marsh   34.5  \nevery time i look at someone i see flashes o...
8  mitchell   34.0  that would be nice i'd like to use the restroo...
9  ridgeway   19.0  \ni didn't really think i had a a urge to kill...


## Enrichment

In [4]:
df = pd.read_pickle("gold_psy.pkl")

In [5]:
handbook = ["ridgeway", "bundy", "kemper", "gacy", "dahmer"]
cbc = ["bernardo"]
articles = ["mitchell", "marsh"]
doc = ["wayne", "carl"]

group_dict = {name: 'handbook' for name in handbook}
group_dict.update({name: 'cbc' for name in cbc})
group_dict.update({name: 'articles' for name in articles})
group_dict.update({name: 'doc' for name in doc})

In [6]:
group_dict

{'ridgeway': 'handbook',
 'bundy': 'handbook',
 'kemper': 'handbook',
 'gacy': 'handbook',
 'dahmer': 'handbook',
 'bernardo': 'cbc',
 'mitchell': 'articles',
 'marsh': 'articles',
 'wayne': 'doc',
 'carl': 'doc'}

In [7]:
df

name  score                                               text
0     wayne   40.0  \nmy very first contemplation of criminal beha...
1      carl   36.0  I had a statutory rape on a minor fifteen uh t...
2     bundy   34.0  \nyes that's true\n\ni grew up in a wonderful ...
3  bernardo   35.0  \npaul jason teal aka paul bernardo\n\n\nyeah ...
4    dahmer   23.0  \nhey you're lucky you came up on a\n\n\nday w...
5      gacy   27.0  \n\nwell that's hogwash yeah that's strictly h...
6    kemper   26.0  \nwell i'm not an expert i'm not an authority ...
7     marsh   34.5  \nevery time i look at someone i see flashes o...
8  mitchell   34.0  that would be nice i'd like to use the restroo...
9  ridgeway   19.0  \ni didn't really think i had a a urge to kill...

In [8]:
import re

def clean_text(text):
    # Replace newline characters with a space
    text = text.replace('\n', ' ')

    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)

    # Remove non-alphanumeric characters (but keep single whitespaces)
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)

    # Convert text to lowercase
    return text.lower()


# Example usage

df['clean_text'] = df['text'].apply(clean_text)

In [9]:
def two_tier_score(score):
    if score > 30:
        return "psychopathic"
    else:
        return "not psychopathic"

# Apply the function to the 'Score' column
df['binary'] = df['score'].apply(two_tier_score)

In [10]:
def three_tier_score(score):
    if score >= 30:
        return "high"
    elif 21 <= score < 29:
        return "medium"
    else:
        return "low"

# Apply the function to the 'score' column
df['trinary'] = df['score'].apply(three_tier_score)


In [11]:
def select_rows_by_group(group_name):
    return df[df['group'] == group_name]

df['group'] = df['name'].map(group_dict)

## Track Misses

In [14]:
import pandas as pd
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer

# Initialize LeaveOneOut
loocv = LeaveOneOut()

# Prepare the classifier
clf = LogisticRegression(max_iter=1000)  # Increase max_iter if needed

# Convert text data to numerical data
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(df['clean_text'])
y = df['binary']

# Store the accuracy for each fold, predictions, and misclassified instances details
accuracies = []
all_y_test = []
all_y_pred = []
misclassified_details = []

for train_index, test_index in loocv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_test)

    # Store true labels and predictions
    all_y_test.extend(y_test)
    all_y_pred.extend(y_pred)

    # Check for misclassification
    if y_pred[0] != y_test.iloc[0]:
        misclassified_info = {
            "Name": df['name'].iloc[test_index[0]],
            "TextSnippet": df['clean_text'].iloc[test_index[0]][:50],  # First 50 characters of text
            "ActualLabel": y_test.iloc[0],
            "PredictedLabel": y_pred[0]
        }
        misclassified_details.append(misclassified_info)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

# Calculate the average accuracy
average_accuracy = sum(accuracies) / len(accuracies)
print(f"Average Accuracy: {average_accuracy}")

# Generate classification report
print("Classification Report:\n")
print(classification_report(all_y_test, all_y_pred))

# Print the details of misclassified instances
print("Misclassified Instances:")
for detail in misclassified_details:
    print(f"Name: {detail['Name']}, Text Snippet: \"{detail['TextSnippet']}...\", Actual Label: {detail['ActualLabel']}, Predicted Label: {detail['PredictedLabel']}")



Average Accuracy: 0.8
Classification Report:

                  precision    recall  f1-score   support

not psychopathic       0.75      0.75      0.75         4
    psychopathic       0.83      0.83      0.83         6

        accuracy                           0.80        10
       macro avg       0.79      0.79      0.79        10
    weighted avg       0.80      0.80      0.80        10

Misclassified Instances:
Name: bundy, Text Snippet: " yes thats true i grew up in a wonderful home with...", Actual Label: psychopathic, Predicted Label: not psychopathic
Name: kemper, Text Snippet: " well im not an expert im not an authority im some...", Actual Label: not psychopathic, Predicted Label: psychopathic


#Regression

In [17]:
import pandas as pd
from sklearn.model_selection import LeaveOneOut
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer

# Initialize LeaveOneOut
loocv = LeaveOneOut()

# Prepare the regression model
reg = LinearSVR(max_iter=10000)  # Increase max_iter if needed

# Convert text data to numerical data
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(df['clean_text'])
y = df['score']

# Store the MSE for each fold, predictions, and names
mse_values = []
all_y_test = []
all_y_pred = []
names = []

for train_index, test_index in loocv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    reg.fit(X_train, y_train)

    # Make predictions
    y_pred = reg.predict(X_test)

    # Store true labels, predictions, and names
    all_y_test.extend(y_test)
    all_y_pred.extend(y_pred)
    names.extend(df['name'].iloc[test_index])

    # Calculate MSE
    mse = mean_squared_error(y_test, y_pred)
    mse_values.append(mse)

# Calculate the average MSE and R-squared
average_mse = sum(mse_values) / len(mse_values)
r_squared = r2_score(all_y_test, all_y_pred)
print(f"Average MSE: {average_mse}")
print(f"R-squared: {r_squared}")

# Print the names with actual and predicted scores
print("Predictions:")
for name, actual, predicted in zip(names, all_y_test, all_y_pred):
    print(f"Name: {name}, Actual Score: {actual}, Predicted Score: {predicted}")



Average MSE: 995.5518099011603
R-squared: -23.825180721929065
Predictions:
Name: wayne, Actual Score: 40.0, Predicted Score: 20.077820803253772
Name: carl, Actual Score: 36.0, Predicted Score: 1.7387908665139133
Name: bundy, Actual Score: 34.0, Predicted Score: 53.702180405103505
Name: bernardo, Actual Score: 35.0, Predicted Score: 79.15997724755769
Name: dahmer, Actual Score: 23.0, Predicted Score: 67.8683875412462
Name: gacy, Actual Score: 27.0, Predicted Score: 20.015096487562946
Name: kemper, Actual Score: 26.0, Predicted Score: 61.0412166713122
Name: marsh, Actual Score: 34.5, Predicted Score: -0.17823994710876423
Name: mitchell, Actual Score: 34.0, Predicted Score: 14.75903346931513
Name: ridgeway, Actual Score: 19.0, Predicted Score: 53.40743899589537


In [18]:
import pandas as pd
from sklearn.model_selection import LeaveOneOut
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer

# Initialize LeaveOneOut
loocv = LeaveOneOut()

# Prepare the KNN regression model
# You can adjust the number of neighbors (n_neighbors) as needed
reg = KNeighborsRegressor(n_neighbors=5)

# Convert text data to numerical data
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(df['clean_text'])
y = df['score']

# Store the MSE for each fold, predictions, and names
mse_values = []
all_y_test = []
all_y_pred = []
names = []

for train_index, test_index in loocv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    reg.fit(X_train, y_train)

    # Make predictions
    y_pred = reg.predict(X_test)

    # Store true labels, predictions, and names
    all_y_test.extend(y_test)
    all_y_pred.extend(y_pred)
    names.extend(df['name'].iloc[test_index])

    # Calculate MSE
    mse = mean_squared_error(y_test, y_pred)
    mse_values.append(mse)

# Calculate the average MSE and R-squared
average_mse = sum(mse_values) / len(mse_values)
r_squared = r2_score(all_y_test, all_y_pred)
print(f"Average MSE: {average_mse}")
print(f"R-squared: {r_squared}")

# Print the names with actual and predicted scores
print("Predictions:")
for name, actual, predicted in zip(names, all_y_test, all_y_pred):
    print(f"Name: {name}, Actual Score: {actual}, Predicted Score: {predicted}")


Average MSE: 28.702999999999992
R-squared: 0.28425908609188966
Predictions:
Name: wayne, Actual Score: 40.0, Predicted Score: 32.9
Name: carl, Actual Score: 36.0, Predicted Score: 33.7
Name: bundy, Actual Score: 34.0, Predicted Score: 32.3
Name: bernardo, Actual Score: 35.0, Predicted Score: 31.1
Name: dahmer, Actual Score: 23.0, Predicted Score: 29.2
Name: gacy, Actual Score: 27.0, Predicted Score: 30.4
Name: kemper, Actual Score: 26.0, Predicted Score: 33.9
Name: marsh, Actual Score: 34.5, Predicted Score: 34.0
Name: mitchell, Actual Score: 34.0, Predicted Score: 32.4
Name: ridgeway, Actual Score: 19.0, Predicted Score: 28.9


In [32]:
import pandas as pd
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer

# Initialize LeaveOneOut
loocv = LeaveOneOut()

# Prepare the ElasticNet regression model
# Adjust alpha for overall strength and l1_ratio for the balance between L1 and L2
reg = ElasticNet(alpha=1.0, l1_ratio=0.5, max_iter=10000)

# Convert text data to numerical data
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(df['clean_text'])
y = df['score']

# Store the MSE for each fold, predictions, and names
mse_values = []
all_y_test = []
all_y_pred = []
names = []

for train_index, test_index in loocv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    reg.fit(X_train, y_train)

    # Make predictions
    y_pred = reg.predict(X_test)

    # Store true labels, predictions, and names
    all_y_test.extend(y_test)
    all_y_pred.extend(y_pred)
    names.extend(df['name'].iloc[test_index])

    # Calculate MSE
    mse = mean_squared_error(y_test, y_pred)
    mse_values.append(mse)

# Calculate the average MSE and R-squared
average_mse = sum(mse_values) / len(mse_values)
r_squared = r2_score(all_y_test, all_y_pred)
print(f"Average MSE: {average_mse}")
print(f"R-squared: {r_squared}")

# Print the names with actual and predicted scores
print("Predictions:")
for name, actual, predicted in zip(names, all_y_test, all_y_pred):
    print(f"Name: {name}, Actual Score: {actual}, Predicted Score: {predicted}")


Average MSE: 14.680759408103654
R-squared: 0.6339190971110615
Predictions:
Name: wayne, Actual Score: 40.0, Predicted Score: 34.143965877985195
Name: carl, Actual Score: 36.0, Predicted Score: 43.89732401392467
Name: bundy, Actual Score: 34.0, Predicted Score: 36.74072237263068
Name: bernardo, Actual Score: 35.0, Predicted Score: 34.736880051519414
Name: dahmer, Actual Score: 23.0, Predicted Score: 26.760499025800655
Name: gacy, Actual Score: 27.0, Predicted Score: 26.419547746091652
Name: kemper, Actual Score: 26.0, Predicted Score: 25.054994807251475
Name: marsh, Actual Score: 34.5, Predicted Score: 34.03624611388423
Name: mitchell, Actual Score: 34.0, Predicted Score: 39.092155671049916
Name: ridgeway, Actual Score: 19.0, Predicted Score: 20.024456455525026


In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer
from scipy.stats import uniform

# Load your data into a DataFrame named df
# Assuming df['clean_text'], df['score'], and df['name'] are already defined

# Convert text data to numerical data
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['clean_text'])
y = df['score']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define parameter distribution for RandomizedSearchCV
param_dist = {
    'alpha': uniform(0.0001, 10),  # Uniform distribution over alpha
   # Uniform distribution over l1_ratio
}

# Initialize RandomizedSearchCV with ElasticNet model, parameter distribution, and 10-fold CV
random_search = RandomizedSearchCV(ElasticNet(max_iter=10000), param_distributions=param_dist,
                                   n_iter=100, cv=5, scoring='neg_mean_squared_error', random_state=42)

# Fit RandomizedSearchCV on the training data
random_search.fit(X_train, y_train)

# Get the best model from RandomizedSearchCV
best_model = random_search.best_estimator_

# Make predictions using the best model on the test set
y_pred = best_model.predict(X_test)

# Calculate MSE and R-squared for the test set
mse = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)
print(f"Mean Squared Error on Test Set: {mse}")
print(f"R-squared on Test Set: {r_squared}")

# Get the best hyperparameters from RandomizedSearchCV
best_params = random_search.best_params_
print("Best Hyperparameters:", best_params)


Mean Squared Error on Test Set: 28.8285838704027
R-squared on Test Set: -27.8285838704027
Best Hyperparameters: {'alpha': 1.0790142699330445}


In [30]:
import pandas as pd
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.feature_extraction.text import CountVectorizer

# Load your data into a DataFrame named df
# Assuming df['clean_text'], df['score'], and df['name'] are already defined

# Convert text data to numerical data
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['clean_text'])
y = df['score']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize LeaveOneOut and Linear Regression model
loocv = LeaveOneOut()
model = LinearRegression()

# Perform LOOCV
mse_scores = cross_val_score(model, X_train, y_train, cv=loocv, scoring=make_scorer(mean_squared_error))
r2_scores = cross_val_score(model, X_train, y_train, cv=loocv, scoring='r2')

# Calculate average MSE and R-squared
average_mse = mse_scores.mean()
average_r2 = r2_scores.mean()
print(f"Average MSE (LOOCV): {average_mse}")
print(f"Average R-squared (LOOCV): {average_r2}")

# Fit the model on the entire training set and evaluate on the test set
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
test_r2 = r2_score(y_test, y_pred)
print(f"Test Set MSE: {test_mse}")
print(f"Test Set R-squared: {test_r2}")


Average MSE (LOOCV): 18.90603211841362
Average R-squared (LOOCV): nan
Test Set MSE: 33.382226629639504
Test Set R-squared: -32.382226629639504


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, Undefined

In [31]:
import pandas as pd
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.feature_extraction.text import CountVectorizer

# Load your data into a DataFrame named df
# Assuming df['clean_text'], df['score'], and df['name'] are already defined

# Convert text data to numerical data
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['clean_text'])
y = df['score']

# Initialize LeaveOneOut and Linear Regression model
loocv = LeaveOneOut()
model = LinearRegression()

# Perform LOOCV for MSE
mse_scores = cross_val_score(model, X, y, cv=loocv, scoring=make_scorer(mean_squared_error))
average_mse = mse_scores.mean()

# Perform LOOCV for R-squared
r2_scores = cross_val_score(model, X, y, cv=loocv, scoring='r2')
average_r2 = r2_scores.mean()

print(f"Average MSE (LOOCV): {average_mse}")
print(f"Average R-squared (LOOCV): {average_r2}")


Average MSE (LOOCV): 25.360891599475167
Average R-squared (LOOCV): nan


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, Undefined

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer

# Assuming df['clean_text'], df['score'], and df['name'] are already defined in the DataFrame df

# Convert text data to numerical data
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['clean_text'])
y = df['score']

# Split the dataset into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

# Prepare the ElasticNet regression model
reg = ElasticNet(alpha=1.0, l1_ratio=0.5, max_iter=10000)

# Train the model
reg.fit(X_train, y_train)

# Make predictions
y_pred = reg.predict(X_test)

# Calculate MSE and R-squared for the test set
test_mse = mean_squared_error(y_test, y_pred)
test_r_squared = r2_score(y_test, y_pred)
print(f"Test Set MSE: {test_mse}")
print(f"Test Set R-squared: {test_r_squared}")

# Print the names with actual and predicted scores
names_test = df['name'].iloc[y_test.index]
print("Predictions:")
for name, actual, predicted in zip(names_test, y_test, y_pred):
    print(f"Name: {name}, Actual Score: {actual}, Predicted Score: {predicted}")


Test Set MSE: 69.54954384927954
Test Set R-squared: -0.6891834689540011
Predictions:
Name: mitchell, Actual Score: 34.0, Predicted Score: 29.93089720327533
Name: carl, Actual Score: 36.0, Predicted Score: 25.051056084611915
Name: gacy, Actual Score: 27.0, Predicted Score: 27.39352749461512
Name: wayne, Actual Score: 40.0, Predicted Score: 25.003279075845903
Name: marsh, Actual Score: 34.5, Predicted Score: 28.468460973113487
Name: bundy, Actual Score: 34.0, Predicted Score: 26.876158996155986
Name: ridgeway, Actual Score: 19.0, Predicted Score: 25.18261760627703


## Random Forest

In [23]:
import pandas as pd
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer

# Initialize LeaveOneOut
loocv = LeaveOneOut()

# Prepare the Random Forest regression model
reg = RandomForestRegressor(n_estimators=100, random_state=42)  # n_estimators can be adjusted

# Convert text data to numerical data
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['clean_text'])
y = df['score']

# Store the MSE for each fold, predictions, and names
mse_values = []
all_y_test = []
all_y_pred = []
names = []

for train_index, test_index in loocv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    reg.fit(X_train, y_train)

    # Make predictions
    y_pred = reg.predict(X_test)

    # Store true labels, predictions, and names
    all_y_test.extend(y_test)
    all_y_pred.extend(y_pred)
    names.extend(df['name'].iloc[test_index])

    # Calculate MSE
    mse = mean_squared_error(y_test, y_pred)
    mse_values.append(mse)

# Calculate the average MSE and R-squared
average_mse = sum(mse_values) / len(mse_values)
r_squared = r2_score(all_y_test, all_y_pred)
print(f"Average MSE: {average_mse}")
print(f"R-squared: {r_squared}")

# Print the names with actual and predicted scores
print("Predictions:")
for name, actual, predicted in zip(names, all_y_test, all_y_pred):
    print(f"Name: {name}, Actual Score: {actual}, Predicted Score: {predicted}")


Average MSE: 41.049947499999995
R-squared: -0.02362564678012591
Predictions:
Name: wayne, Actual Score: 40.0, Predicted Score: 32.5
Name: carl, Actual Score: 36.0, Predicted Score: 33.22
Name: bundy, Actual Score: 34.0, Predicted Score: 30.95
Name: bernardo, Actual Score: 35.0, Predicted Score: 30.185
Name: dahmer, Actual Score: 23.0, Predicted Score: 31.54
Name: gacy, Actual Score: 27.0, Predicted Score: 31.735
Name: kemper, Actual Score: 26.0, Predicted Score: 30.755
Name: marsh, Actual Score: 34.5, Predicted Score: 31.52
Name: mitchell, Actual Score: 34.0, Predicted Score: 32.6
Name: ridgeway, Actual Score: 19.0, Predicted Score: 32.61


In [26]:
import pandas as pd
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer

# Initialize LeaveOneOut
loocv = LeaveOneOut()

# Prepare the SGD Regressor model with elasticnet penalty
reg = SGDRegressor(penalty='elasticnet', alpha=0.0001, l1_ratio=0.5, max_iter=1000, tol=0.0001)

# Convert text data to numerical data
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['clean_text'])
y = df['score']

# Store the MSE for each fold, predictions, and names
mse_values = []
all_y_test = []
all_y_pred = []
names = []

for train_index, test_index in loocv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    reg.fit(X_train, y_train)

    # Make predictions
    y_pred = reg.predict(X_test)

    # Store true labels, predictions, and names
    all_y_test.extend(y_test)
    all_y_pred.extend(y_pred)
    names.extend(df['name'].iloc[test_index])

    # Calculate MSE
    mse = mean_squared_error(y_test, y_pred)
    mse_values.append(mse)

# Calculate the average MSE and R-squared
average_mse = sum(mse_values) / len(mse_values)
r_squared = r2_score(all_y_test, all_y_pred)
print(f"Average MSE: {average_mse}")
print(f"R-squared: {r_squared}")

# Print the names with actual and predicted scores
print("Predictions:")
for name, actual, predicted in zip(names, all_y_test, all_y_pred):
    print(f"Name: {name}, Actual Score: {actual}, Predicted Score: {predicted}")


Average MSE: 2.606972415801099e+27
R-squared: -6.500772809179226e+25
Predictions:
Name: wayne, Actual Score: 40.0, Predicted Score: -14009391689429.361
Name: carl, Actual Score: 36.0, Predicted Score: -798805124152.9541
Name: bundy, Actual Score: 34.0, Predicted Score: -77624255804567.55
Name: bernardo, Actual Score: 35.0, Predicted Score: -46731373409087.17
Name: dahmer, Actual Score: 23.0, Predicted Score: -28507699652136.8
Name: gacy, Actual Score: 27.0, Predicted Score: -447497971266.27655
Name: kemper, Actual Score: 26.0, Predicted Score: 56727730569774.08
Name: marsh, Actual Score: 34.5, Predicted Score: 67874116993868.43
Name: mitchell, Actual Score: 34.0, Predicted Score: 36346121217392.93
Name: ridgeway, Actual Score: 19.0, Predicted Score: 87775940714026.5


## Experiments

In [58]:
df["length"] = df["clean_text"].apply(lambda x: len(x.split()))
copy = df

In [60]:
copy

name  score                                               text  \
0     wayne   40.0  \nmy very first contemplation of criminal beha...   
1      carl   36.0  I had a statutory rape on a minor fifteen uh t...   
2     bundy   34.0  \nyes that's true\n\ni grew up in a wonderful ...   
3  bernardo   35.0  \npaul jason teal aka paul bernardo\n\n\nyeah ...   
4    dahmer   23.0  \nhey you're lucky you came up on a\n\n\nday w...   
5      gacy   27.0  \n\nwell that's hogwash yeah that's strictly h...   
6    kemper   26.0  \nwell i'm not an expert i'm not an authority ...   
7     marsh   34.5  \nevery time i look at someone i see flashes o...   
8  mitchell   34.0  that would be nice i'd like to use the restroo...   
9  ridgeway   19.0  \ni didn't really think i had a a urge to kill...   

                                          clean_text            binary  \
0   my very first contemplation of criminal behav...      psychopathic   
1  i had a statutory rape on a minor fifteen uh t...      psychopathic   
2   yes thats true i grew up in a wonderful home ...      psychopathic   
3   paul jason teal aka paul bernardo yeah im and...      psychopathic   
4   hey youre lucky you came up on a day when the...  not psychopathic   
5   well thats hogwash yeah thats strictly hogwas...  not psychopathic   
6   well im not an expert im not an authority im ...  not psychopathic   
7   every time i look at someone i see flashes of...      psychopathic   
8  that would be nice id like to use the restroom...      psychopathic   
9   i didnt really think i had a a urge to kill h...  not psychopathic   

  trinary     group  length  
0    high       doc     581  
1    high       doc      84  
2    high  handbook    1508  
3    high       cbc    1720  
4  medium  handbook    1553  
5  medium  handbook    1575  
6  medium  handbook    1581  
7    high  articles    1263  
8    high  articles    1214  
9     low  handbook    1666

## Binary - LOOCV

In [61]:
import pandas as pd
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer

# Initialize LeaveOneOut
loocv = LeaveOneOut()

# Prepare the classifier
clf = LogisticRegression(max_iter=1000)  # Increase max_iter if needed

# Convert text data to numerical data
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(df['clean_text'])
y = df['binary']

# Store the accuracy for each fold and predictions
accuracies = []
all_y_test = []
all_y_pred = []

for train_index, test_index in loocv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_test)

    # Store true labels and predictions
    all_y_test.extend(y_test)
    all_y_pred.extend(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

# Calculate the average accuracy
average_accuracy = sum(accuracies) / len(accuracies)
print(f"Average Accuracy: {average_accuracy}")

# Generate classification report
print("Classification Report:\n")
print(classification_report(all_y_test, all_y_pred))




Average Accuracy: 0.8
Classification Report:

                  precision    recall  f1-score   support

not psychopathic       0.75      0.75      0.75         4
    psychopathic       0.83      0.83      0.83         6

        accuracy                           0.80        10
       macro avg       0.79      0.79      0.79        10
    weighted avg       0.80      0.80      0.80        10



## Binary - KFold

In [62]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer

# Initialize KFold
kf = KFold(n_splits=5)

# Prepare the classifier
clf = LogisticRegression()

# Convert text data to numerical data
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['clean_text'])
y = df['binary']

# Store the accuracy for each fold and predictions
accuracies = []
all_y_test = []
all_y_pred = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_test)

    # Store true labels and predictions
    all_y_test.extend(y_test)
    all_y_pred.extend(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

# Calculate the average accuracy
average_accuracy = sum(accuracies) / len(accuracies)
print(f"Average Accuracy: {average_accuracy}")

# Generate classification report
print("Classification Report:\n")
print(classification_report(all_y_test, all_y_pred))




Average Accuracy: 0.7
Classification Report:

                  precision    recall  f1-score   support

not psychopathic       0.67      0.50      0.57         4
    psychopathic       0.71      0.83      0.77         6

        accuracy                           0.70        10
       macro avg       0.69      0.67      0.67        10
    weighted avg       0.70      0.70      0.69        10



## TTS - Group Comparison (but have to use 'Handbook' in train because of healthy controls)

In [21]:
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline

# Function to train with Stratified K-Fold CV based on group, and return the trained model
def train_and_return_model_skf(df, group_names, k=2):
    # Filter DataFrame based on the list of groups
    group_df = df[df['group'].isin(group_names)]

    # Split data into features (X) and target (y)
    X = group_df['clean_text']
    y = group_df['binary']

    # Initialize StratifiedKFold
    skf = StratifiedKFold(n_splits=k)

    accuracies = []

    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))

    average_accuracy = sum(accuracies) / len(accuracies)
    return pipeline, average_accuracy

# Example usage
group_name = ['articles', 'handbook']
model, accuracy = train_and_return_model_skf(df, group_name)
print(f"Average Accuracy for training groups '{group_name}' with Stratified K-Fold CV: {accuracy}")



NameError: ignored

In [42]:
def get_texts_by_group(df, group_names):
    # Filter DataFrame based on the list of groups
    filtered_df = df[df['group'].isin(group_names)]

    # Return the 'clean_text' column from the filtered DataFrame
    return filtered_df['clean_text'].tolist()

# Example usage with a list of group names
test_names = ['doc', 'cbc']
test_texts = get_texts_by_group(df, test_names)
print(test_texts)


[' my very first contemplation of criminal behavior was trying to uh put together a plan of getting away with murder and i was gonna kill my brother i knew that every sunday id jump on his bike and hed head down the road to join up with some other fellas and he loved that bike and he used to tell me you ever touch my bike ill kill you so this sunday morning while hes working i take his bike i knew if i had provoked it just a little bit more that he would chase me so i gave him a couple of kicks and took off on a dead run for a tree that was right beside my grandfathers barn and it went up onto the roof and on the opposite side of the barn there were cutter rakes and sure enough he chased me up there and as he uh topped edge of that roof i kicked him and he fell onto those rakes some court people came got me i stabbed my first man um i uh stabbed him uh he lived but it sent out a word a clear word to the rest of them that uh you dont wanna be messing with this kid hell stick ya this par

In [23]:
predictions = model.predict(test_texts)
print(predictions)

NameError: ignored

## Trinary - LOOCV

In [63]:
import pandas as pd
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer

# Initialize LeaveOneOut
loocv = LeaveOneOut()

# Prepare the classifier
clf = LogisticRegression(max_iter=1000)  # Increase max_iter if needed

# Convert text data to numerical data
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(df['clean_text'])
y = df['trinary']

# Store the accuracy for each fold and predictions
accuracies = []
all_y_test = []
all_y_pred = []

for train_index, test_index in loocv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_test)

    # Store true labels and predictions
    all_y_test.extend(y_test)
    all_y_pred.extend(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

# Calculate the average accuracy
average_accuracy = sum(accuracies) / len(accuracies)
print(f"Average Accuracy: {average_accuracy}")

# Generate classification report
print("Classification Report:\n")
print(classification_report(all_y_test, all_y_pred))




Average Accuracy: 0.7
Classification Report:

              precision    recall  f1-score   support

        high       0.83      0.83      0.83         6
         low       0.00      0.00      0.00         1
      medium       0.50      0.67      0.57         3

    accuracy                           0.70        10
   macro avg       0.44      0.50      0.47        10
weighted avg       0.65      0.70      0.67        10



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Trinary - Kfold

In [64]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer

# Initialize KFold
kf = KFold(n_splits=5)

# Prepare the classifier
clf = LogisticRegression()

# Convert text data to numerical data
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['clean_text'])
y = df['trinary']  # Change this to the column with three categories

# Store the accuracy for each fold and predictions
accuracies = []
all_y_test = []
all_y_pred = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_test)

    # Store true labels and predictions
    all_y_test.extend(y_test)
    all_y_pred.extend(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

# Calculate the average accuracy
average_accuracy = sum(accuracies) / len(accuracies)
print(f"Average Accuracy: {average_accuracy}")

# Generate classification report
print("Classification Report:\n")
print(classification_report(all_y_test, all_y_pred))


Average Accuracy: 0.5
Classification Report:

              precision    recall  f1-score   support

        high       0.62      0.83      0.71         6
         low       0.00      0.00      0.00         1
      medium       0.00      0.00      0.00         3

    accuracy                           0.50        10
   macro avg       0.21      0.28      0.24        10
weighted avg       0.38      0.50      0.43        10



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

## TTS - All

In [65]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer

# Load your dataset (assuming df is already loaded)
# df = pd.read_csv('your_dataset.csv')

# Convert text data to numerical data
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['clean_text'])
y = df['binary']

# Split the data into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the hyperparameter grid for the LogisticRegression model
param_dist = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'max_iter': [100, 500, 1000, 1500],
}

# Initialize the classifier
clf = LogisticRegression()

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    clf, param_distributions=param_dist, n_iter=10, cv=4, scoring='accuracy', random_state=42)

# Fit RandomizedSearchCV to find the best hyperparameters
random_search.fit(X_train, y_train)

# Get the best estimator with the optimized hyperparameters
best_clf = random_search.best_estimator_

# Train the best model on the full training set
best_clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on the test set: {accuracy}")

# Generate classification report
print("Classification Report:\n")
print(classification_report(y_test, y_pred))


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(


Accuracy on the test set: 1.0
Classification Report:

                  precision    recall  f1-score   support

not psychopathic       1.00      1.00      1.00         1
    psychopathic       1.00      1.00      1.00         2

        accuracy                           1.00         3
       macro avg       1.00      1.00      1.00         3
    weighted avg       1.00      1.00      1.00         3

